# Update Program Code Descriptions

In [2]:
import numpy as np
import pandas as pd
from siuba import *

import dla_utils
from calitp_data_analysis.sql import to_snakecase

import utils

In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [ ]:
df = utils.read_data_all()
    
    ## function that adds known agency name to df 
df = utils.identify_agency(df, 'summary_recipient_defined_text_field_1_value')

In [ ]:
df.sample(3)

In [ ]:
#checking to make sure summary_recipient_defined_text_field_1_value (and implementing_agency) is unique
df>>group_by(_.project_number)>>summarize(n = _.summary_recipient_defined_text_field_1_value.nunique())>>arrange(-_.n)

### Condense Projects into One

In [ ]:
df_agg = df.astype(str).groupby('project_number').agg(' | ' .join).reset_index()

In [ ]:
df_agg.sample()

In [ ]:
(list(df.columns))

In [ ]:
df.info()

In [ ]:
df2 = df.copy()

In [ ]:
df_agg = utils.condense_df(df2)

In [ ]:
df_agg.sample(5)

In [ ]:
#should return 552
len(df_agg)

In [ ]:
# def condense_df(df):
    
#     # make sure columns are in string format
#     df[['county_code', 'improvement_type',
#      'implementing_agency_locode', 'district',
#      'program_code_description', 'recipient_project_number']] = df[['county_code', 'improvement_type',
#                                                                      'implementing_agency_locode', 'district',
#                                                                      'program_code_description', 'recipient_project_number']].astype(str)
#     # aggreate df using .agg function and join in the unique values into one row
#     df_agg = (df
#            .assign(count=1)
#            .groupby(['fmis_transaction_date','project_number', 'implementing_agency', 'summary_recipient_defined_text_field_1_value'])
#            .agg({'program_code':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'program_code_description':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'recipient_project_number':lambda x:' | '.join(x.unique()), #'first',
#                  'improvement_type':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'improvement_type_description':lambda x:' | '.join(x.unique()),  # get unique values to concatenate
#                  'project_title':'first', #should be the same                 
#                  'obligations_amount':'sum', #sum of the obligations amount
#                  'congressional_district':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'district':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'county_code':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'county_name':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'implementing_agency_locode':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'rtpa_name':'first', #should be the same
#                  'mpo_name':'first',  #should be the same
#                 }).reset_index())
    
#     return df_agg

In [ ]:
# df2[['county_code'
#      , 'improvement_type',
#      'implementing_agency_locode',
#      'district',
#      'program_code_description',
#      'recipient_project_number']] = df2[['county_code',
#                                          'improvement_type',
#                                          'implementing_agency_locode',
#                                          'district', 'program_code_description',
#                                          'recipient_project_number']].astype(str)

In [ ]:
# df_agg = (df2
#            .assign(count=1)
#            .groupby(['fmis_transaction_date','project_number', 'implementing_agency', 'summary_recipient_defined_text_field_1_value'])
#            .agg({'program_code':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'program_code_description':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'recipient_project_number':lambda x:' | '.join(x.unique()), #'first',
#                  'improvement_type':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'improvement_type_description':lambda x:' | '.join(x.unique()),  # get unique values to concatenate
#                  'project_title':'first', #should be the same                 
#                  'obligations_amount':'sum', #sum of the obligations amount
#                  'congressional_district':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'district':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'county_code':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'county_name':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'implementing_agency_locode':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'rtpa_name':'first', #should be the same
#                  'mpo_name':'first',  #should be the same
#                 }).reset_index())

In [ ]:
len(df_agg>>filter(_.improvement_type.str.contains(' | ')))

In [ ]:
full_df_agg = utils.get_new_desc_title(df_agg)

In [ ]:
full_df_agg

### New issue: `project_title_new` taking groupings from `improvement_type_description`


In [ ]:
#checking the description column to add "project" flag and subsetting data
test_df = (utils.add_description(full_df_agg, 'project_title'))>>filter(_.project_type=='Project')
# test_df = (test_df>>select(_.implementing_agency, _.program_code_description, _.improvement_type, _.improvement_type_description, _.project_title,
#                           _.county_name, _.project_title_new, _.project_type))

In [ ]:
len(test_df)

In [ ]:
print((f"There are {len(test_df>>filter(_.improvement_type.str.contains(' | ')))} "
           f"entires out of {len(full_df_agg)} that have concatenated entired for improvement_type_description"))

In [ ]:
#entries with concatenated improvement types descriptions
(test_df>>filter(_.improvement_type.str.contains(' | '))).improvement_type_description.value_counts()

In [ ]:
## filter down to just the projects with multiple improvement type descriptions
test_df = (test_df>>filter(_.improvement_type.str.contains(' | ')))

In [ ]:
# #function to rework DOES NOT WORK
# def update_no_matched(df, flag_col, desc_col, program_code_desc_col): 
#     """
#     function to itreate over projects that did not match the first time
#     using an existing project's short description of project type. 
#     """
    
#     def return_project_type(df):
        
#         if (df[flag_col] == "Project") & df[desc_col].str.contains("Bridge Rehabilitation") | df[desc_col].str.contains("Bridge Rehabilitation - No Added Capacity") | df[desc_col].str.contains("Bridge Rehabilitation - Added Capacity"):
#             return ("Bridge Rehabilitation")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Facilities for Pedestrians and Bicycles"):
#             return (df[desc_col])
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Safety"):
#             return (df[desc_col] + " Improvements")
            
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Planning "):
#             return "Project Planning" 
            
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Preliminary Engineering"):
#             return (df[desc_col] + " Projects ")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Construction Engineering"):
#             return (df[desc_col] + " Projects")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("4R - Restoration & Rehabilitation"):
#             return ("Road Restoration & Rehabilitation")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("4R - Maintenance  Resurfacing"):
#             return ("Maintenance Resurfacing")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Bridge Replacement - Added Capacity")| df[desc_col].str.contains("Bridge Replacement - No Added Capacity") | df[desc_col].str.contains("Bridge New Construction")| df[desc_col].str.contains("Special Bridge"):
#             return ("Bridge Replacement")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Mitigation of Water Pollution due to Highway Runoff"):
#             return (df[desc_col])
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("4R - Added Capacity"):
#             return ("Added Roadway Capacity")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("4R - No Added Capacity"):
#             return ("Road Construction")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("New  Construction Roadway"):
#             return ("New Construction Roadway")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Traffic Management/Engineering - HOV"):
#             return ("Traffic Management Project")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Right of Way"):
#             return (df[desc_col] + " Project")
        
#         # elif (df[flag_col] == "Project") & (df[program_code_desc_col]== "National Highway Performance Program (NHPP)"): 
#         #     return ("National Highway Performance Program Support") 
        
#         # elif (df[flag_col] == "Project") & (df[desc_col] != "Other"):
#         #     return (df[desc_col])
    
#         else:
#             return df[flag_col] 

#         return df

#     df['project_type2'] = df.apply(return_project_type, axis = 1)

#     return df

In [ ]:
#test_df2 = update_no_matched(test_df, 'project_type', 'improvement_type_description', 'program_code_description')

In [ ]:
#test_df2.sample(5)

In [ ]:
#trying another version
#update: worked-- moving to utils
def add_description_4_no_match(df, desc_col):
    ##using np.where. code help: https://stackoverflow.com/questions/43905930/conditional-if-statement-if-value-in-row-contains-string-set-another-column

    
    ## method for project in first column
    df['project_type2'] = (np.where(df[desc_col].str.contains("Bridge Rehabilitation"),"Bridge Rehabilitation",
                        np.where(df[desc_col].str.contains("Bridge Rehabilitation - No Added Capacity") | df[desc_col].str.contains("Bridge Rehabilitation - Added Capacity"), "Bridge Rehabilitation",
                        np.where(df[desc_col].str.contains("Bridge Replacement - Added Capacity")| df[desc_col].str.contains("Bridge Replacement - No Added Capacity"), "Bridge Replacement",
                        np.where(df[desc_col].str.contains("Bridge New Construction")| df[desc_col].str.contains("Special Bridge"), "Bridge Replacement",
                        np.where(df[desc_col].str.contains("Facilities for Pedestrians and Bicycles"), "Facilities for Pedestrians and Bicycles",
                        np.where(df[desc_col].str.contains("Mitigation of Water Pollution due to Highway Runoff"), "Mitigation of Water Pollution due to Highway Runoff",
                        np.where(df[desc_col].str.contains("Traffic Management/Engineering - HOV"), "Traffic Management Project",
                        np.where(df[desc_col].str.contains("Planning "), "Project Planning",
                        np.where(df[desc_col].str.contains("4R - Restoration & Rehabilitation"), "Road Restoration & Rehabilitation",
                        np.where(df[desc_col].str.contains("4R - Maintenance  Resurfacing"), "Maintenance Resurfacing",
                        np.where(df[desc_col].str.contains("4R - Added Capacity"), "Added Roadway Capacity",
                        np.where(df[desc_col].str.contains("4R - No Added Capacity"), "Road Construction",
                        np.where(df[desc_col].str.contains("Safety"), "Safety Improvements",
                        np.where(df[desc_col].str.contains("New  Construction Roadway"), "New Construction Roadway",
                        np.where(df[desc_col].str.contains("Preliminary Engineering"), "Preliminary Engineering Projects",
                        np.where(df[desc_col].str.contains("Construction Engineering"), "Construction Engineering Projects",
                        np.where(df[desc_col].str.contains("Right of Way"), "Right of Way Project",
                                    "Project"))))))))))))))))))
    
    return df

In [ ]:
test_df3 = add_description_4_no_match(test_df, 'improvement_type_description')

In [ ]:
test_df3.project_type2.value_counts()

In [ ]:
test_df3.loc[0:60]

In [ ]:
test_df3>>filter(_.improvement_type_description.str.contains("Other"))

### Add new function to larger function

In [ ]:
# using df_agg

df = utils.condense_df(df)

In [ ]:
proj_unique_cat = utils.add_description(df, 'project_title')

In [ ]:
len(proj_unique_cat>>filter(_.project_type=='Project'))

In [ ]:
proj_unique_cat.loc[proj_unique_cat['project_type'] == 'Project', 'project_method'] = ""

In [ ]:
proj_unique_cat['project_type'] = proj_unique_cat['project_type'].replace('Project', np.NaN)

In [ ]:
proj_unique_cat_title =  utils.add_description_4_no_match(proj_unique_cat, 'improvement_type_description')


In [ ]:
proj_unique_cat_title['project_type'] = proj_unique_cat_title['project_type'].fillna(proj_unique_cat_title['project_type2'])


In [ ]:
proj_unique_cat_title = utils.add_new_title(proj_unique_cat, "project_method", "project_type", "implementing_agency", "county_name")


In [ ]:
proj_unique_cat_title = proj_unique_cat_title.rename(columns={'project_name_new':'project_title_new'})
#proj_unique_cat_title.drop(columns =['project_method', 'project_type', 'project_type2'], axis=1, inplace=True)

In [ ]:
proj_title_mapping = (dict(proj_unique_cat_title[['project_number', 'project_title_new']].values))

In [ ]:
df['project_title_new'] = df.project_number.map(proj_title_mapping)

In [ ]:
df

In [ ]:
# from get_new_desc_title
    
    
#     proj_unique_cat = add_description(proj_unique, 'project_title')
    
#     #remove project method column values so that the title function wont double count
#     proj_unique_cat.loc[proj_unique_cat['project_type'] == 'Project', 'project_method'] = ""
    # proj_unique_cat['project_type'].replace('Project', np.NaN)
    
    #update for the projects not in the first round of descriptions
    # proj_unique_cat_title =  add_description_4_no_match(proj_unique_cat, 'improvement_type_description')
    ## proj_unique_cat_title = update_no_matched(proj_unique_cat, "project_type", 'improvement_type_description', 'program_code_description')
    
    #fill nan values in 'Project_type' with values from 'project_type2' from add_description_4_no_match function
    # proj_unique_cat_title['project_type'] = proj_unique_cat_title['project_type'].fillna(proj_unique_cat_title['project_type2'])
    
    #add title - second round to account for statewide projects
    # proj_unique_cat_title = add_new_title(proj_unique_cat, "project_method", "project_type", "implementing_agency", "county_name")
    
    # rename new title one
    # proj_unique_cat_title = proj_unique_cat_title.rename(columns={'project_name_new':'project_title_new'})
    # proj_unique_cat_title.drop(columns =['project_method', 'project_type', 'project_type2'], axis=1, inplace=True)
    
    #map the title back to df
#     proj_title_mapping = (dict(proj_unique_cat_title[['project_number', 'project_title_new']].values))
    
#     df['project_title_new'] = df.project_number.map(proj_title_mapping)

#     return df

## Test & Export

In [4]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [5]:
###test agg. PASS df with title column, concat and 552 rows
agg = utils.get_clean_data(full_or_agg = 'agg')

In [ ]:
sorted(list(agg.project_title_new))

In [6]:
agg.sample(5)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,project_title_new
219,2022-06-14,5060377,Fresno,L5060COFCG,Y400,Congestion Mitigation & Air Quality Improvement (CMAQ),0621000057L,24.0,Traffic Management/Engineering - HOV,"JENSEN AVE. FROM ELM AVENUE TO TEMPERANCE AVENUE INSTALL ITS ADAPTIVE SYSTEM, UPGRADE DETECTION AND SYNCHRONIZE CORRIDOR. (TC)",1057300.00,Cong Dist 16 | Cong Dist 22,6.0,19.0,Fresno County,5060.0,Council of Fresno County Governments,Council Of Fresno County Goverments,Install Synchronize Corridor in Fresno
309,2022-07-18,5937207,Santa Clara County,L5937MTC,Y233,Surface Transportation Block Grant (STBG) Program,0417000077L,15.0 | 16.0,Preliminary Engineering | Right of Way,ALDERCROFT HEIGHTS ROAD OVER LOS GATOS CREEK 2.38 MILES SOUTH OF LOS GATOS SC ROAD. BRIDGE 37C0508 REPLACE EXISTING ONE-LANE BRIDGE WITH A NEW 2-LANE,330479.53,Cong Dist 18,4.0,85.0,Santa Clara County,5937.0,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Replace Bridge in Santa Clara County
408,2022-09-01,2152252,California,S SCAG,YS30,Highway Safety Improvement Program (HSIP),0819000045S,17.0 | 21.0 | 42.0,Construction Engineering | Safety | Training,IN RIVERSIDE COUNTY NEAR MORENO VALLEY ON INTERSTATE 215 FROM 0.2 MILE NORTH OF ALESSANDRO BOULEVARD OVERCROSSING TO 0.2 MILE SOUTH OF EUCALYPTUS AVE,5867900.00,Cong Dist 41,8.0,65.0,Riverside County,nan,None,None,Safety Improvements in Riverside County
3,2022-02-09,5117016,Hercules,L5117MTC,Y230,Surface Transportation Block Grant (STBG) Program,0420000346L,5.0,4R - Maintenance Resurfacing,HERCULES: SYCAMORE AVE FROM CIVIC DR TO WILLOW/PALM AVE PAVEMENT REHABILITATION,492000.00,Cong Dist 5,4.0,13.0,Contra Costa County,5117.0,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Pavement Rehabilitation in Hercules
148,2022-05-09,P002041,California,S SCAG,YS30,Highway Safety Improvement Program (HSIP),0716000270S,17.0 | 21.0 | 42.0,Construction Engineering | Safety | Training,IN LOS ANGELES COUNTY FROM BAY TREE RD TO BIG PINES HWYINSTALL AND UPGRADE MBGR THE PROJECT PROPOSES TO UPGRADE THE EXISTING NON-STANDARD METAL BEAM,20039900.00,Cong Dist 27 | Cong Dist 28,7.0,37.0,Los Angeles County,nan,None,None,Safety Improvements in Los Angeles County


In [7]:
agg = utils.title_column_names(agg)

In [11]:
len(agg)

552

In [14]:
agg.head(1)

,Fmis Transaction Date,Project Number,Implementing Agency,Summary Recipient Defined Text Field 1 Value,Program Code,Program Code Description,Recipient Project Number,Improvement Type,Improvement Type Description,Project Title,Obligations Amount,Congressional District,District,County Code,County Name,Implementing Agency Locode,Rtpa Name,Mpo Name,Project Title New
0,2022-02-04,5064095,Pasadena,L5064SCAG,YS30,Highway Safety Improvement Program (HSIP),0721000154L,17.0 | 21.0,Construction Engineering | Safety,VARIOUS LOCATIONS THROUGHOUT THE WEST SIDE OF THE CITY. REPAIR EXISTING DAMAGED GUARDRAI,289000.0,Cong Dist 27,7.0,37.0,Los Angeles County,5064.0,Los Angeles County Metropolitan Transportation Auth.,Southern California Association Of Governments,Repair Guardrails in Pasadena


In [17]:
agg.District.dtypes

dtype('O')

In [18]:
agg.District.value_counts()

4.0           101
3.0            81
10.0           57
7.0            53
8.0            50
6.0            45
nan            41
12.0           29
5.0            23
11.0           23
1.0            17
2.0            17
9.0            10
7.0 | 8.0       2
7.0 | 12.0      1
1.0 | 11.0      1
4.0 | 5.0       1
Name: District, dtype: int64

In [8]:
# agg.to_csv(f"{GCS_FILE_PATH}/FMIS_projects_agg.csv")

In [9]:
###test full. PASS title, no concat and 1241 rows
full = utils.get_clean_data(full_or_agg = 'full')

In [10]:
full = utils.title_column_names(full)

In [ ]:
# full.to_csv(f"{GCS_FILE_PATH}/FMIS_projects_all.csv")

In [12]:
len(full)

1241